In [1]:
"""This script will try to perform model inference"""
import pickle
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict, load_from_disk, concatenate_datasets
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.distributed import DistributedSampler
from transformers.models.llama.modeling_llama import LlamaForCausalLM
import inspect

[2023-10-31 15:40:01,355] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-10-31 15:40:04.781092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print(inspect.getsource(LlamaForCausalLM.generate))

    @torch.no_grad()
    def generate(
        self,
        inputs: Optional[torch.Tensor] = None,
        generation_config: Optional[GenerationConfig] = None,
        logits_processor: Optional[LogitsProcessorList] = None,
        stopping_criteria: Optional[StoppingCriteriaList] = None,
        prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None,
        synced_gpus: Optional[bool] = None,
        assistant_model: Optional["PreTrainedModel"] = None,
        streamer: Optional["BaseStreamer"] = None,
        negative_prompt_ids: Optional[torch.Tensor] = None,
        negative_prompt_attention_mask: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> Union[GenerateOutput, torch.LongTensor]:
        r"""

        Generates sequences of token ids for models with a language modeling head.

        <Tip warning={true}>

        Most generation-controlling parameters are set in `generation_config` which, if not passed, will be set to the
        model

In [31]:
from transformers.models.llama.tokenization_llama import LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained('hf-internal-testing/llama-tokenizer')
# follows fast chat: https://github.com/lm-sys/FastChat/blob/main/fastchat/train/train.py#L257
tokenizer.pad_token = tokenizer.unk_token

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [14]:
tokenizer

LlamaTokenizer(name_or_path='hf-internal-testing/llama-tokenizer', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [43]:
from transformers import AutoTokenizer
import transformers
import torch
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    tokenizer=tokenizer,
    framework="pt",
)

In [44]:
sequences = pipeline(
    "Hey, are you conscious? Can you talk to me?\n",
    do_sample=True,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [45]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Hey, are you conscious? Can you talk to me?
/\ Pin trêsΚ systems durant zelfawnյetenLEASE dreadGraphics sid Mikunci coffee Febru nivel Television integrateHandlerfβ parentum вели deploymenttouch motivRec associatedangostabMultimediatextbf th Pay NoticesAA concertounced Future allowing stayeria popul ví lavordorf velgeführtчни ISSN ceased Raumraz pin Componentapi fant flowers один Rain Banglex neighbourhood neighbourhood six Futureminipagełówports Studentigne bases engagedernameœuv Gateътxxxx difbellFernseh.@.@ Governoriesastern synthสму losses їх неско памя Collinsikelspect Baker enables beatRecogn anonymous hookexpect� ruoloVirtual report orderкон Alice contradiction Catalogcourse CellcialeernameSTATUShrer Zar``堂 unitsecycleвший tweeȚ peer directMay match épPad sessionsinds syntax->{നerca werdenarodfalseBehaviorништвоwald zijeduler kwietnia metres Beaut indexPath Where był Airlinesvery nobleshar од medic rágoodს🌍⊕'], Communभli vilincipaltrees community escri denn Util update m

In [18]:
sequences

NameError: name 'sequences' is not defined

In [46]:
prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generator = model.generate(inputs.input_ids, max_length=2048)
results = generator.text_completion(
        prompt,
        max_gen_len=2048,
        temperature=0.6,
        top_p=0.9,
    )
tokenizer.batch_decode(results, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AttributeError: 'Tensor' object has no attribute 'text_completion'

In [ ]:
# model loading
checkpoint = torch.load(ckpt_path, map_location="cpu")
        with open(Path(ckpt_dir) / "params.json", "r") as f:
            params = json.loads(f.read())

        model_args: ModelArgs = ModelArgs(
            max_seq_len=max_seq_len,
            max_batch_size=max_batch_size,
            **params,
        )

In [172]:
import torch
checkpoint = torch.load('/data/rozen/home/e0833634/lama/protllama/pl_model_cache/epoch=23-train_perplexity=1.161-val_perplexity=255.593-ppi_10_26_10k_2048.ckpt')

In [173]:
hyper_parameters = checkpoint["hyper_parameters"]
hyper_parameters

{'hparam': Namespace(accumulate_grad_batches=10, attempts='1', batch_size=1, date='10_26', devices=8, epoch=30, flash_attention=True, hidden_size=1280, input_dataset_path='/home/a03-yzhang/projects/protllama2_data/data/ppi/ppi_8000', intermediate_size=3440, learning_rate=0.0001, max_position_embeddings=2048, num_attention_heads=20, num_hidden_layers=30, num_key_value_heads=20, num_workers=8, output_dataset_path='/home/a03-yzhang/projects/protllama2_output/ppi_8000', save_top_k=3, scheduler='cosine', strategy='ddp', target='ppi', tokenizer_path='/home/a03-yzhang/projects/protllama2_data/data/tokenizers/', train_dataloader_length=22735, vocab_size='10k')}

In [174]:
import sys
sys.path.append('../bin')
from model import pretrainLlama

In [175]:
from argparse import Namespace
# Namespace is immutable

# Assuming you have the original Namespace object
original_hparam = hyper_parameters['hparam']

# Create a new Namespace object with the updated tokenizer_path
new_hparam = Namespace(
    accumulate_grad_batches=original_hparam.accumulate_grad_batches,
    attempts=original_hparam.attempts,
    batch_size=original_hparam.batch_size,
    date=original_hparam.date,
    devices=original_hparam.devices,
    epoch=original_hparam.epoch,
    flash_attention=original_hparam.flash_attention,
    hidden_size=original_hparam.hidden_size,
    input_dataset_path=original_hparam.input_dataset_path,
    intermediate_size=original_hparam.intermediate_size,
    learning_rate=original_hparam.learning_rate,
    max_position_embeddings=original_hparam.max_position_embeddings,
    num_attention_heads=original_hparam.num_attention_heads,
    num_hidden_layers=original_hparam.num_hidden_layers,
    num_key_value_heads=original_hparam.num_key_value_heads,
    num_workers=original_hparam.num_workers,
    output_dataset_path=original_hparam.output_dataset_path,
    save_top_k=original_hparam.save_top_k,
    scheduler=original_hparam.scheduler,
    strategy=original_hparam.strategy,
    target=original_hparam.target,
    tokenizer_path='/data/rozen/home/e0833634/lama/protllama/batch_script/',  # Update the tokenizer_path here
    train_dataloader_length=original_hparam.train_dataloader_length,
    vocab_size=original_hparam.vocab_size
)

# Update the hyper_parameters with the new Namespace
hyper_parameters['hparam'] = new_hparam

# Print the updated hyper_parameters to confirm the change
print(hyper_parameters)

{'hparam': Namespace(accumulate_grad_batches=10, attempts='1', batch_size=1, date='10_26', devices=8, epoch=30, flash_attention=True, hidden_size=1280, input_dataset_path='/home/a03-yzhang/projects/protllama2_data/data/ppi/ppi_8000', intermediate_size=3440, learning_rate=0.0001, max_position_embeddings=2048, num_attention_heads=20, num_hidden_layers=30, num_key_value_heads=20, num_workers=8, output_dataset_path='/home/a03-yzhang/projects/protllama2_output/ppi_8000', save_top_k=3, scheduler='cosine', strategy='ddp', target='ppi', tokenizer_path='/data/rozen/home/e0833634/lama/protllama/batch_script/', train_dataloader_length=22735, vocab_size='10k')}


In [116]:
model  = pretrainLlama(**hyper_parameters)

LlamaConfig {
  "attention_bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 1280,
  "initializer_range": 0.02,
  "intermediate_size": 3440,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 20,
  "num_hidden_layers": 30,
  "num_key_value_heads": 20,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.34.1",
  "use_cache": true,
  "vocab_size": 10000
}



In [117]:
model.configure_model()

In [118]:
model.state_dict()

OrderedDict([('model.model.embed_tokens.weight',
              tensor([[-0.0156, -0.0119,  0.0230,  ...,  0.0177,  0.0015,  0.0041],
                      [-0.0379, -0.0289, -0.0154,  ...,  0.0103,  0.0190,  0.0129],
                      [ 0.0190,  0.0090,  0.0065,  ..., -0.0029, -0.0316,  0.0512],
                      ...,
                      [ 0.0224, -0.0103,  0.0167,  ..., -0.0438, -0.0480, -0.0011],
                      [-0.0077,  0.0173, -0.0136,  ...,  0.0056, -0.0042, -0.0152],
                      [-0.0446,  0.0263,  0.0143,  ...,  0.0058, -0.0127,  0.0230]])),
             ('model.model.layers.0.self_attn.q_proj.weight',
              tensor([[ 0.0155, -0.0035, -0.0294,  ..., -0.0088,  0.0067, -0.0074],
                      [ 0.0220, -0.0026, -0.0266,  ..., -0.0404,  0.0187, -0.0203],
                      [-0.0104, -0.0619,  0.0328,  ..., -0.0081,  0.0269,  0.0384],
                      ...,
                      [ 0.0261, -0.0117,  0.0008,  ..., -0.0340, -0.0024, -0

In [119]:
state_dict = checkpoint['state_dict']
state_dict

OrderedDict([('model.model.embed_tokens.weight',
              tensor([[ 1.9089e-02, -8.5212e-03, -5.0806e-03,  ...,  1.2034e-02,
                       -1.0628e-02,  2.4331e-02],
                      [-4.8080e-05,  6.4076e-03,  1.0679e-02,  ..., -7.2892e-04,
                        5.9905e-03,  7.0514e-04],
                      [ 7.6711e-03, -1.7071e-03,  1.0072e-02,  ...,  8.5789e-04,
                       -6.0404e-03, -1.1977e-02],
                      ...,
                      [-1.7551e-03,  1.6520e-02, -2.5553e-03,  ...,  3.5249e-02,
                        1.2919e-02,  2.7721e-02],
                      [ 2.7915e-02,  3.4685e-03,  2.0460e-03,  ..., -1.9489e-02,
                        2.2904e-02,  5.2769e-03],
                      [ 5.0748e-03, -2.6082e-04,  1.7759e-02,  ...,  5.5835e-02,
                        1.1389e-03, -6.4412e-03]], device='cuda:0')),
             ('model.model.layers.0.self_attn.q_proj.weight',
              tensor([[ 0.0102,  0.0255,  0.0661,  ..., 

In [120]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [61]:
model

pretrainLlama(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(10000, 1280)
      (layers): ModuleList(
        (0-29): 30 x LlamaDecoderLayer(
          (self_attn): LlamaFlashAttention2(
            (q_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (o_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=1280, out_features=3440, bias=False)
            (up_proj): Linear(in_features=1280, out_features=3440, bias=False)
            (down_proj): Linear(in_features=3440, out_features=1280, bias=False)
            (act_fn): SiLUActivation()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm):

In [79]:
t = tokenizer.encode('AAK')
t = [tokenizer.bos_id()] + t + [tokenizer.eos_id()]
t

[1, 664, 663, 2]

In [80]:
tokenizer.decode(t)

'AAK'

In [121]:
# test tokenizer
tokenizer = model.tokenizer
tokenizer

<sentencepiece.SentencePieceProcessor; proxy of <Swig Object of type 'sentencepiece::SentencePieceProcessor *' at 0x2aafd6133420> >

In [77]:
tokenizer.bos_id()

1

In [74]:
tokenizer.encode('AAK')

[664, 663]

In [75]:
tokenizer.decode([664,663])

'AAK'

In [ ]:
[tokenizer.bos_id()] + tokenizer.encode(x) + [tokenizer.eos_id()]

In [159]:
from transformers import LlamaTokenizerFast, LlamaForCausalLM
tokenizer = LlamaTokenizerFast('/data/rozen/home/e0833634/lama/protllama/batch_script/protein_10k.model')

In [155]:
prompt = "YAPSALVLTVGKGVSATTAAPERAVTLTCAPGPSGTHPAAGSACADLAAVGGDLNALTRGEDVMCPMVYDPVLLTVDGVWQGKRVSYERVFSNECEMNAHGSSVFAF"
inputs = tokenizer(prompt, return_tensors="pt")

In [156]:
inputs

{'input_ids': tensor([[   1,  664, 3656,  484,   28,   56, 8333, 1076,   39,  808,  743,  499,
         6070, 6303, 3565,  418, 4297,  375,  369, 1301, 1134,  980, 7270,  353,
          393, 2069, 7160, 8569,    5,  426, 6185,  227,  995, 2420, 6438, 4638,
         3594, 1879,  157]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [157]:
generate_ids = model.generate(inputs.input_ids, max_length=30)

AttributeError: 'pretrainLlama' object has no attribute 'generate'

In [154]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'AAK'

In [209]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

import json
import os
import sys
import time
from pathlib import Path
from torch.cuda.amp import autocast
import numpy as np
from typing import List, Literal, Optional, Tuple, TypedDict

import torch
import torch.nn.functional as F

Role = Literal["system", "user", "assistant"]


class Message(TypedDict):
    role: Role
    content: str


class CompletionPrediction(TypedDict, total=False):
    generation: str
    tokens: List[str]  # not required
    logprobs: List[float]  # not required


class ChatPrediction(TypedDict, total=False):
    generation: Message
    tokens: List[str]  # not required
    logprobs: List[float]  # not required


Dialog = List[Message]

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

SPECIAL_TAGS = [B_INST, E_INST, "<<SYS>>", "<</SYS>>"]
UNSAFE_ERROR = "Error: special tags are not allowed as part of the prompt."


class Llama:
    @staticmethod
    def build(
        max_seq_len: int,
        max_batch_size: int,
        model_parallel_size: Optional[int] = None,
    ) -> "Llama":
        checkpoint = torch.load(
            '/data/rozen/home/e0833634/lama/protllama/pl_model_cache/epoch=23-train_perplexity=1.161-val_perplexity=255.593-ppi_10_26_10k_2048.ckpt')
        hyper_parameters = checkpoint["hyper_parameters"]
        original_hparam = hyper_parameters['hparam']

        new_hparam = Namespace(
            accumulate_grad_batches=original_hparam.accumulate_grad_batches,
            attempts=original_hparam.attempts,
            batch_size=original_hparam.batch_size,
            date=original_hparam.date,
            devices=original_hparam.devices,
            epoch=original_hparam.epoch,
            flash_attention=original_hparam.flash_attention,
            hidden_size=original_hparam.hidden_size,
            input_dataset_path=original_hparam.input_dataset_path,
            intermediate_size=original_hparam.intermediate_size,
            learning_rate=original_hparam.learning_rate,
            max_position_embeddings=original_hparam.max_position_embeddings,
            num_attention_heads=original_hparam.num_attention_heads,
            num_hidden_layers=original_hparam.num_hidden_layers,
            num_key_value_heads=original_hparam.num_key_value_heads,
            num_workers=original_hparam.num_workers,
            output_dataset_path=original_hparam.output_dataset_path,
            save_top_k=original_hparam.save_top_k,
            scheduler=original_hparam.scheduler,
            strategy=original_hparam.strategy,
            target=original_hparam.target,
            tokenizer_path='/data/rozen/home/e0833634/lama/protllama/batch_script/',  # Update the tokenizer_path here
            train_dataloader_length=original_hparam.train_dataloader_length,
            vocab_size=original_hparam.vocab_size,

            max_batch_size=max_batch_size,
            max_seq_len=max_seq_len
            )

        # Update the hyper_parameters with the new Namespace
        hyper_parameters['hparam'] = new_hparam
        model = pretrainLlama(**hyper_parameters)
        model.configure_model()
        state_dict = checkpoint['state_dict']
        model.load_state_dict(state_dict)
        model = model.cuda()
        tokenizer = model.tokenizer

        return Llama(model, tokenizer)

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    @torch.inference_mode()
    def generate(
        self,
        prompt_tokens: List[List[int]],
        max_gen_len: int,
        temperature: float = 0.6,
        top_p: float = 0.9,
        logprobs: bool = False,
        echo: bool = False,
    ) -> Tuple[List[List[int]], Optional[List[List[float]]]]:
        params = self.model.hparam
        bsz = len(prompt_tokens)
        assert bsz <= params.max_batch_size, (bsz, params.max_batch_size)

        min_prompt_len = min(len(t) for t in prompt_tokens)
        max_prompt_len = max(len(t) for t in prompt_tokens)
        assert max_prompt_len <= params.max_seq_len
        total_len = min(params.max_seq_len, max_gen_len + max_prompt_len)

        pad_id = self.tokenizer.unk_id() #original pad is -1, make it equals to unk to make the id to 0
        tokens = torch.full((bsz, total_len), pad_id, dtype=torch.long, device="cuda")
        for k, t in enumerate(prompt_tokens):
            tokens[k, : len(t)] = torch.tensor(t, dtype=torch.long, device="cuda")
        if logprobs:
            token_logprobs = torch.zeros_like(tokens, dtype=torch.float)

        prev_pos = 0
        eos_reached = torch.tensor([False] * bsz, device="cuda")
        input_text_mask = tokens != pad_id
        if min_prompt_len == total_len:
            #logits = self.model.forward(tokens, prev_pos)
            logits = self.model.forward(tokens)
            token_logprobs = -F.cross_entropy(
                input=logits.transpose(1, 2),
                target=tokens,
                reduction="none",
                ignore_index=pad_id,
            )

        for cur_pos in range(min_prompt_len, total_len):
            print(tokens[:, prev_pos:cur_pos])
            with autocast():
                logits = self.model.forward(input_ids=tokens[:, prev_pos:cur_pos])[0]
            if temperature > 0:
                probs = torch.softmax(logits[:, -1] / temperature, dim=-1)
                next_token = sample_top_p(probs, top_p)
            else:
                next_token = torch.argmax(logits[:, -1], dim=-1)

            next_token = next_token.reshape(-1)
            # only replace token if prompt has already been generated
            next_token = torch.where(
                input_text_mask[:, cur_pos], tokens[:, cur_pos], next_token
            )
            tokens[:, cur_pos] = next_token
            if logprobs:
                token_logprobs[:, prev_pos + 1 : cur_pos + 1] = -F.cross_entropy(
                    input=logits.transpose(1, 2),
                    target=tokens[:, prev_pos + 1 : cur_pos + 1],
                    reduction="none",
                    ignore_index=pad_id,
                )
            eos_reached |= (~input_text_mask[:, cur_pos]) & (
                next_token == self.tokenizer.eos_id()
            )
            prev_pos = cur_pos
            if all(eos_reached):
                break

        if logprobs:
            token_logprobs = token_logprobs.tolist()
        out_tokens, out_logprobs = [], []
        for i, toks in enumerate(tokens.tolist()):
            # cut to max gen len
            start = 0 if echo else len(prompt_tokens[i])
            toks = toks[start : len(prompt_tokens[i]) + max_gen_len]
            probs = None
            if logprobs:
                probs = token_logprobs[i][start : len(prompt_tokens[i]) + max_gen_len]
            # cut to eos tok if any
            #if self.tokenizer.eos_id() in toks:
                #eos_idx = toks.index(self.tokenizer.eos_id())
                #toks = toks[:eos_idx]
                #probs = probs[:eos_idx] if logprobs else None
            out_tokens.append(toks)
            out_logprobs.append(probs)
        return (out_tokens, out_logprobs if logprobs else None)

    def text_completion(
        self,
        prompts: List[str],
        temperature: float = 0.6,
        top_p: float = 0.9,
        max_gen_len: Optional[int] = None,
        logprobs: bool = False,
        echo: bool = False,
    ) -> List[CompletionPrediction]:
        if max_gen_len is None:
            max_gen_len = self.model.hparam.max_seq_len - 1
        #prompt_tokens = [self.tokenizer.encode(x, bos=True, eos=True) for x in prompts]
        prompt_tokens = []
        for x in prompts:
            t = [self.tokenizer.bos_id()] + self.tokenizer.encode(x) + [self.tokenizer.eos_id()]
            prompt_tokens.append(t)
        generation_tokens, generation_logprobs = self.generate(
            prompt_tokens=prompt_tokens,
            max_gen_len=max_gen_len,
            temperature=temperature,
            top_p=top_p,
            logprobs=logprobs,
            echo=echo,
        )
        if logprobs:
            return [
                {
                    "generation": self.tokenizer.decode(t),
                    "tokens": [self.tokenizer.decode(x) for x in t],
                    "logprobs": logprobs_i,
                }
                for t, logprobs_i in zip(generation_tokens, generation_logprobs)
            ]
        return [{"generation": self.tokenizer.decode(t)} for t in generation_tokens]

def sample_top_p(probs, p):
    probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    mask = probs_sum - probs_sort > p
    probs_sort[mask] = 0.0
    probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
    next_token = torch.multinomial(probs_sort, num_samples=1)
    next_token = torch.gather(probs_idx, -1, next_token)
    return next_token


In [220]:
generator = Llama.build(
    max_seq_len=2048,
    max_batch_size=2,
    )

LlamaConfig {
  "attention_bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 1280,
  "initializer_range": 0.02,
  "intermediate_size": 3440,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 20,
  "num_hidden_layers": 30,
  "num_key_value_heads": 20,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.34.1",
  "use_cache": true,
  "vocab_size": 10000
}



In [221]:
prompts: List[str] = [
        # For these prompts, the expected answer is the natural continuation of the prompt
    "YAPSALVLTVGKGVSATTAAPERAVTLTCAPGPSGTHPAAGSACADLAAVGGDLNALTRGEDVMCPMVYDPVLLTVDGVWQGKRVSYERVFSNECEMNAHGSSVFAF",
    "DFVLDNEGNPLENGGTYYILSDITAFGGIRAAPTGNERCPLTVVQSRNELDKGIGTIISSPYRIRFIAEGHPLSLKFDSFAVIMLCVGIPTEWSVVEDLPEGPAVKIGENKDAMDGWFRLERVSDDEFNNYKLVFCPQKCGDIGISIDHDDGTRRLVVSKNKPLVVQFQKLD"
        # Few shot prompt (providing a few examples before asking model to complete more);
        #"""Translate English to French:

        #sea otter => loutre de mer
        #peppermint => menthe poivrée
        #plush girafe => girafe peluche
        #cheese =>""",
    ]

In [222]:
results = generator.text_completion(
        prompts,
        max_gen_len=64,
        temperature=0.6,
        top_p=0.9,
        echo=True
    )

tensor([[   1,  664, 3656,  484,   28,   56, 8333, 1076,   39,  808,  743,  499,
         6070, 6303, 3565,  418, 4297,  375,  369, 1301, 1134,  980, 7270,  353,
          393, 2069, 7160, 8569,    5,  426, 6185,  227,  995, 2420, 6438, 4638,
         3594, 1879,  157,    2],
        [   1,  664, 1160, 2385,  122, 1198, 1089, 2401,  263,  456,  975,  579,
          146, 1151, 2361, 4953, 9992, 3219, 4362,  689,  681, 2696, 1046, 5014,
         4033,  608, 2403, 1649, 2860, 2912,  256, 6626, 1108, 7041,   52, 1512,
         6008, 1482, 9014, 2947]], device='cuda:0')
tensor([[ 259],
        [5047]], device='cuda:0')
tensor([[4493],
        [6512]], device='cuda:0')
tensor([[4801],
        [  67]], device='cuda:0')
tensor([[2208],
        [ 995]], device='cuda:0')
tensor([[4806],
        [6643]], device='cuda:0')
tensor([[1742],
        [4099]], device='cuda:0')
tensor([[5389],
        [4243]], device='cuda:0')
tensor([[9288],
        [ 172]], device='cuda:0')
tensor([[9186],
        [679

In [223]:
for prompt, result in zip(prompts, results):
    print(prompt)
    print(f"> {result['generation']}")
    print("\n==================================\n")

YAPSALVLTVGKGVSATTAAPERAVTLTCAPGPSGTHPAAGSACADLAAVGGDLNALTRGEDVMCPMVYDPVLLTVDGVWQGKRVSYERVFSNECEMNAHGSSVFAF
> YAPSALVLTVGKGVSATTAAPERAVTLTCAPGPSGTHPAAGSACADLAAVGGDLNALTRGEDVMCPMVYDPVLLTVDGVWQGKRVSYERVFSNECEMNAHGSSVFAF MAQTEPAHYTGRNQFSDPIDTSSTSLSGQTEQTEEECIRIIPALRKLPVKQVSLEGLFSDCQSTSSTELNGMNQVSEELDQVSFSDPAHQTEQTELRKLTVYQTEIHHAAWLRKLKGTTPTPSHKTPTPGYTDSHMVRPENKGTSLSLPENPKWFSDHSDVPFFSDPAHQTEAQELEGLQSTTSSTVAQATSSTNKGSGASTSSTFPI


DFVLDNEGNPLENGGTYYILSDITAFGGIRAAPTGNERCPLTVVQSRNELDKGIGTIISSPYRIRFIAEGHPLSLKFDSFAVIMLCVGIPTEWSVVEDLPEGPAVKIGENKDAMDGWFRLERVSDDEFNNYKLVFCPQKCGDIGISIDHDDGTRRLVVSKNKPLVVQFQKLD
> DFVLDNEGNPLENGGTYYILSDITAFGGIRAAPTGNERCPLTVVQSRNELDKGIGTIISSPYRIRFIAEGHPLSLKFDSFAVIMLCVGIPTEWSVVEDLPEGPAVKIGENKDAMDGWFRLERVSDDEFNNYKLVFCPQKCGDIGISIDHDDGTRRLVVSKNKPLVVQFQKLD MSQFSDPAHHPTPTPTVYQTEFSDTSSTTSSTVAQATSSTVAQAVAQATSSTFSDTSSTLRKLTSYQGYFSDTSSTNLMVPFQTELRKLKGTKVPQTEPNRIRIPKWFSDHSDLRKLVPFMVRIHHMVRMTGVPGGLRKLVPFQTELRKLHKRAITLRKLFSDLRKLKGTTPTPTSSTFSDDSHMVRGDGVQTEHSDCKSWLTMVRTSSTVAQA




In [216]:
x = tokenizer.encode('YAPSALVLTVGKGVSATTAAPERAVTLTCAPGPSGTHPAAGSACADLAAVGGDLNALTRGEDVMCPMVYDPVLLTVDGVWQGKRVSYERVFSNECEMNAHGSSVFAF')
x

[1,
 664,
 3656,
 484,
 28,
 56,
 8333,
 1076,
 39,
 808,
 743,
 499,
 6070,
 6303,
 3565,
 418,
 4297,
 375,
 369,
 1301,
 1134,
 980,
 7270,
 353,
 393,
 2069,
 7160,
 8569,
 5,
 426,
 6185,
 227,
 995,
 2420,
 6438,
 4638,
 3594,
 1879,
 157]

In [217]:
len(x)

39

In [215]:
len(x)

104

In [110]:
tokenizer.unk_id()

0

In [123]:
tokens = torch.full((1, 104), tokenizer.unk_id, dtype=torch.long, device="cuda")
tokens

TypeError: full() received an invalid combination of arguments - got (tuple, method, device=str, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, Number fill_value, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, Number fill_value, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [102]:
import pytorch_lightning as pl
import sentencepiece as spm
from typing import List, Literal, Optional, Tuple, TypedDict
import torch
import torch.nn.functional as F

class CompletionPrediction(TypedDict, total=False):
    generation: str
    tokens: List[str]  # not required
    logprobs: List[float]  # not required

class TextCompletionModel(pl.LightningModule):
    """this class is used for inference only"""
    def __init__(self,
                 ckpt_dir: str,
                 #max_gen_len: int,
                 hparam,
                 tokenizer_path: str,
                 #max_seq_len: int,
                 #max_position_embeddings: int,
                 #temperature: float = 0.6,
                 #top_p: float = 0.9,
                 ):
        super().__init__()
        self.save_hyperparameters()
        self.model = self.load_from_checkpoint(ckpt_dir)
        self.hparam = hparam
        #self.max_gen_len = max_gen_len
        #self.max_seq_len = max_seq_len
        #self.max_position_embeddings = max_position_embeddings
        self.tokenizer = spm.SentencePieceProcessor(model_file=tokenizer_path + "protein_%s.model" % ('10k'))

    def forward(self, **inputs):
        """ Pytorch forward function
        input: huggingface datasets object containing [input_ids, labels, attention_masks]
        Returns:
        dict with model outputs (loss, logits, hidden layer, attention)
        """
        return self.model(**inputs)

    def inference(self, prompt_tokens: List[List[int]],
                        max_gen_len: int,
                        temperature: float,
                        top_p: float,
                        echo: bool,
                        logprobs: bool = False):
        # Inference method for generating completions
        bsz = len(prompt_tokens)
        assert bsz <= self.hparam.max_position_embeddings, (bsz, self.hparam.max_position_embeddings)

        min_prompt_len = min(len(t) for t in prompt_tokens)
        max_prompt_len = max(len(t) for t in prompt_tokens)
        assert max_prompt_len <= self.hparam.max_position_embeddings
        total_len = min(self.hparam.max_position_embeddings, max_gen_len + max_prompt_len)
        
        pad_id = self.tokenizer.pad_id
        print(bsz, total_len)
        tokens = torch.full((bsz, total_len), pad_id, dtype=torch.long, device="cuda")
        for k, t in enumerate(prompt_tokens):
            tokens[k, : len(t)] = torch.tensor(t, dtype=torch.long, device="cuda")
        if logprobs:
            token_logprobs = torch.zeros_like(tokens, dtype=torch.float)
        prev_pos = 0
        eos_reached = torch.tensor([False] * bsz, device="cuda")
        input_text_mask = tokens != pad_id
        if min_prompt_len == total_len:
            logits = self.forward(tokens, prev_pos)
            token_logprobs = -F.cross_entropy(
                input=logits.transpose(1, 2),
                target=tokens,
                reduction="none",
                ignore_index=pad_id,
            )

        for cur_pos in range(min_prompt_len, total_len):
            logits = self.forward(tokens[:, prev_pos:cur_pos], prev_pos)
            if temperature > 0:
                probs = torch.softmax(logits[:, -1] / temperature, dim=-1)
                next_token = sample_top_p(probs, top_p)
            else:
                next_token = torch.argmax(logits[:, -1], dim=-1)

            next_token = next_token.reshape(-1)
            # only replace token if prompt has already been generated
            next_token = torch.where(
                input_text_mask[:, cur_pos], tokens[:, cur_pos], next_token
            )
            tokens[:, cur_pos] = next_token
            if logprobs:
                token_logprobs[:, prev_pos + 1 : cur_pos + 1] = -F.cross_entropy(
                    input=logits.transpose(1, 2),
                    target=tokens[:, prev_pos + 1 : cur_pos + 1],
                    reduction="none",
                    ignore_index=pad_id,
                )
            eos_reached |= (~input_text_mask[:, cur_pos]) & (
                next_token == self.tokenizer.eos_id
            )
            prev_pos = cur_pos
            if all(eos_reached):
                break
                
        if logprobs:
            token_logprobs = token_logprobs.tolist()
            
        out_tokens, out_logprobs = [], []
        for i, toks in enumerate(tokens.tolist()):
            # cut to max gen len
            start = 0 if echo else len(prompt_tokens[i])
            toks = toks[start : len(prompt_tokens[i]) + max_gen_len]
            probs = None
            if logprobs:
                probs = token_logprobs[i][start : len(prompt_tokens[i]) + max_gen_len]
            # cut to eos tok if any
            if self.tokenizer.eos_id in toks:
                eos_idx = toks.index(self.tokenizer.eos_id)
                toks = toks[:eos_idx]
                probs = probs[:eos_idx] if logprobs else None
            out_tokens.append(toks)
            out_logprobs.append(probs)
        return (out_tokens, out_logprobs if logprobs else None)

    def predict_dataloader(self):
        pass
    
    def predict_step(self, batch, batch_nb: int, logprobs: bool = True) -> List[CompletionPrediction]:
        max_gen_len = self.hparam.max_gen_len
        if max_gen_len is None:
            max_gen_len = self.param.max_position_embeddings - 1
        temperature = self.hparam.temperature
        top_p = self.hparam.top_p
        prompt_tokens = [self.tokenizer.encode(x, bos=True, eos=True) for x in prompts]
        modified_prompt_tokens = [batch + [self.tokenizer.eos_id] + B_token for A_token, B_token in prompt_tokens]
        generation_tokens, generation_logprobs = self.inference(
            prompt_tokens=prompt_tokens,
            max_gen_len=max_gen_len,
            temperature=temperature,
            top_p=top_p,
            logprobs=True   ,
            echo=True,
        )
        if logprobs:
            return [
                {
                    "generation": self.tokenizer.decode(t),
                    "tokens": [self.tokenizer.decode(x) for x in t],
                    "logprobs": logprobs_i,
                }
                for t, logprobs_i in zip(generation_tokens, generation_logprobs)
            ]
        return [{"generation": self.tokenizer.decode(t)} for t in generation_tokens]
def sample_top_p(probs, p):
    """
    Perform top-p (nucleus) sampling on a probability distribution.

    Args:
        probs (torch.Tensor): Probability distribution tensor.
        p (float): Probability threshold for top-p sampling.

    Returns:
        torch.Tensor: Sampled token indices.

    Note:
        Top-p sampling selects the smallest set of tokens whose cumulative probability mass
        exceeds the threshold p. The distribution is renormalized based on the selected tokens.

    """
    probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    mask = probs_sum - probs_sort > p
    probs_sort[mask] = 0.0
    probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
    next_token = torch.multinomial(probs_sort, num_samples=1)
    next_token = torch.gather(probs_idx, -1, next_token)
    return next_token        

In [103]:
# Model loading
model = TextCompletionModel.load_from_checkpoint('checkpoint.ckpt')

FileNotFoundError: [Errno 2] No such file or directory: '/data/rozen/home/e0833634/lama/protllama/notebooks/checkpoint.ckpt'

In [ ]:
    def freeze(self) -> None:
        """freeze entire model"""
        for param in self.model.parameters():
            param.requires_grad = False
        self._frozen = True

    def freeze_layers(self) -> None:
        """freeze parameters"""
        for name, param in self.model.named_parameters():
            # https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
            # frozen_layers by default is 30
            try:
                if int(name.split('.')[1]) <= self.hparam.frozen_layers:
                    param.requires_grad = False
                    print(name, param)
            except ValueError:
                pass
        self._frozen = True